In [1]:
import pandas as pd
import time
import random
import math
import mmh3

In [2]:
def str_to_MinHash(str1, q, seed=0):
    return min([mmh3.hash(str1[i:i + q], seed) for i in range(len(str1) - q + 1)])

def frequent2(temp, L, t):
    return {k: v for (k, v) in temp.items() if v/L >= t}

In [3]:
def topK():
       global tp, fp, pairsNo, dictB, L, q
       for index2 in range(nbS, nbS+offsetB):
            if index2 >= len(df2):
               return True
            rr = df2.iloc[index2, 0:3]
            idScholar = rr["id"]
            title = rr["title"]
            authors = rr["authors"]
            srec = title + " " + authors
            temp = dict()
            indices = [random.randrange(0, L) for i in range(L1)]
            K = 5
            topK = [0] * K
            topKIds = [None] * K
            for l in indices:
                key = str(str_to_MinHash(srec.lower(), 2, l))
                d = dictB[l]
                if key in d:
                    ids = d[key]
                    for id in ids:
                        if id in temp:
                           temp[id] += 1
                        else:
                           temp[id] = 1

            for id in temp.keys():
               if temp[id] / L1 >= t:
                   for i in range(K):
                       if topK[i] <= temp[id]:
                         topK.insert(i, temp[id])
                         topK.pop()
                         topKIds.insert(i, id)
                         topKIds.pop()
                         break

            for id_1 in topKIds:
                if id_1 in truthD:
                    ids = truthD[id_1]
                    if idScholar in ids:
                          tp += 1
                    else:
                          fp += 1
       return False

In [3]:
df1 = pd.read_csv("./copia/DBLP_copy.csv", sep=",", encoding="utf-8", keep_default_na=False)
df2 = pd.read_csv("./copia/Scholar_copy.csv", sep=",", encoding="utf-8", keep_default_na=False)
truth = pd.read_csv("./copia/truth_copy.csv", sep=",", encoding="utf-8", keep_default_na=False)

In [5]:
df1

,id,title,authors,venue,year
0,conf/vldb/RusinkiewiczKTWM95,Towards a Cooperative Transaction Model - The ...,"M Rusinkiewicz, W Klas, T Tesch, J Wδsch, P Muth",VLDB,1995
1,journals/sigmod/EisenbergM02,SQL/XML is Making Good Progress,"A Eisenberg, J Melton",SIGMOD Record,2002
2,conf/vldb/AmmannJR95,Using Formal Methods to Reason about Semantics...,"P Ammann, S Jajodia, I Ray",VLDB,1995
3,conf/sigmod/AbadiCCCCEGHMRSSTXYZ03,Aurora: A Data Stream Management System,"D Abadi, D Carney, U Ηetintemel, M Cherniack, ...",SIGMOD Conference,2003


In [4]:
truthD = dict()
for i, r in truth.iterrows():
    idDBLP = r["idDBLP"]
    idScholar = r["idScholar"]
    if idDBLP in truthD:
        ids = truthD[idDBLP]
        ids.append(idScholar)
    else:
        truthD[idDBLP] = [idScholar]
truthD

{'conf/vldb/RusinkiewiczKTWM95': ['019zSr3Lx4EJ',
  'iWNLOYCQX-YJ',
  'KureLsKbUXYJ',
  'T2fm7Wb1ak4J'],
 'journals/sigmod/EisenbergM02': ['wgK6p4mDSIMJ'],
 'conf/vldb/AmmannJR95': ['x-H7BqZ0Hw8J'],
 'conf/sigmod/AbadiCCCCEGHMRSSTXYZ03': ['eBnT7lhV2LwJ',
  'gBVNSFeS4P8J',
  'VuY9Y49GqXgJ']}

In [5]:
t = 0.5
p1 = (t) ** 8
L = math.ceil(math.log(0.1) / math.log(1 - p1))
# print("Minhash LSH=", L)
TP = 5347
eps = 0.1
w = 1000
delta = 0.1
L = math.ceil(math.log(1 / delta) / (2 * (eps ** 2)))
eps = 0.01
L1 = int(1 / (2 * eps))
print("L=", L, "L1=", L1)
q = 2
dictB = [dict() for l in range(L)]
tp = 0
fp = 0
pairsNo = 0
nbS = 1
naS = 1
offsetA = 1
offsetB = 1
indices = [random.randint(0, L) for i in range(L1)]
blockingTime = 0
matchingTime = 0

L= 116 L1= 50


In [14]:
def matching():
    global tp, fp, pairsNo, L1, q
    for index2 in range(nbS, nbS + offsetB):  # DBLP
        if index2 > len(df2) - 1:
            return True

        rr = df2.iloc[index2, 0:3]
        idScholar = rr["id"]
        title = rr["title"]
        srec = title + " " + rr["authors"]

        temp = dict()
        indices = [random.randrange(0, L) for i in range(L1)]
        matchingPairs = {}
        for l in indices:
            key = str(str_to_MinHash(srec.lower(), q, l))
            d = dictB[l]
            if key in d:
                ids = d[key]
                for id in ids:
                    if id in temp:
                        temp[id] += 1
                        if temp[id] / L1 >= t:
                            matchingPairs[id] = 1
                    else:
                        temp[id] = 1
        for id in matchingPairs.keys():
            idDBLP = id
            pairsNo += 1
            if idDBLP in truthD:
                ids = truthD[idDBLP]
                for id in ids:
                    if id == idScholar:
                        tp += 1
                        break
            else:
                fp += 1

    return False

In [18]:
while True:
    st = time.time()
    for index1 in range(naS, naS + offsetA):  # Scholar
        if index1 >= len(df1):
            print("chegou")
            break
        rr = df1.iloc[index1, 0:3]
        idDBLP = rr["id"]
        title = rr["title"]
        srec = title + " " + rr["authors"]
        key = ""
        print(srec)
        for l in range(L):
            key = str(str_to_MinHash(srec.lower(), 2, l)) #len(key) = 11
            d = dictB[l]
            if key in d:
                ids = d[key]
                if len(ids) < w:
                    ids.append(df1.iloc[index1, 0])
                else:
                    ids.pop(0)
                    ids.append(df1.iloc[index1, 0])
            else:
                d[key] = [df1.iloc[index1, 0]]
    end = time.time()

    blockingTime += (end - st)
    st = time.time()
    termination = matching()  
    end = time.time()
    matchingTime += (end - st)

    if termination:
            break

    nbS += offsetB
    naS += offsetA

chegou


In [16]:
#Execução Padrão, com 
print("offsetA", offsetA, "offsetB", offsetB)
print("blocking time (in mins)", blockingTime / 60)
print("matching time (in mins)", matchingTime / 60)
if tp + fp > 0:
    recall = tp / TP
    precision = tp / (tp + fp)
    print("TP=", tp, "Recall=", recall, "Precision=", precision, "pairsNo=", pairsNo, "F1-Score=", 2 * ((precision * recall)/ (precision + recall)))

offsetA 1 offsetB 1
blocking time (in mins) 0.00027987162272135414
matching time (in mins) 0.00017440319061279297
TP= 5 Recall= 0.0009351037965214139 Precision= 1.0 pairsNo= 5 F1-Score= 0.0018684603886397607


In [ ]:
import logging
import sys

from pyflink.common import Types
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.table import (DataTypes, TableDescriptor, Schema, StreamTableEnvironment)
from pyflink.table.expressions import col
from pyflink.table.udf import udf


def mixing_use_of_datastream_and_table():
    # use StreamTableEnvironment instead of TableEnvironment when mixing use of table & datastream
    env = StreamExecutionEnvironment.get_execution_environment()
    t_env = StreamTableEnvironment.create(stream_execution_environment=env)

    # define the source
    t_env.create_temporary_table(
        'source',
        TableDescriptor.for_connector('datagen')
                       .schema(Schema.new_builder()
                               .column('id', DataTypes.BIGINT())
                               .column('data', DataTypes.STRING())
                               .build())
                       .option("number-of-rows", "10")
                       .build())

    # define the sink
    t_env.create_temporary_table(
        'sink',
        TableDescriptor.for_connector('print')
                       .schema(Schema.new_builder()
                               .column('a', DataTypes.BIGINT())
                               .build())
                       .build())

    @udf(result_type=DataTypes.BIGINT())
    def length(data):
        return len(data)

    # perform table api operations
    table = t_env.from_path("source")
    table = table.select(col('id'), length(col('data')))

    # convert table to datastream and perform datastream api operations
    ds = t_env.to_data_stream(table)
    ds = ds.map(lambda i: i[0] + i[1], output_type=Types.LONG())

    # convert datastream to table and perform table api operations as you want
    table = t_env.from_data_stream(
        ds,
        Schema.new_builder().column("f0", DataTypes.BIGINT()).build())

    # execute
    table.execute_insert('sink') \
         .wait()
    # remove .wait if submitting to a remote cluster, refer to
    # https://nightlies.apache.org/flink/flink-docs-stable/docs/dev/python/faq/#wait-for-jobs-to-finish-when-executing-jobs-in-mini-cluster
    # for more details


if __name__ == '__main__':
    logging.basicConfig(stream=sys.stdout, level=logging.INFO, format="%(message)s")

    mixing_use_of_datastream_and_table()

Using Any for unsupported type: typing.Sequence[~T]
No module named google.cloud.bigquery_storage_v1. As a result, the ReadFromBigQuery transform *CANNOT* be used with `method=DIRECT_READ`.
KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/igor/miniconda3/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/igor/miniconda3/lib/python3.11/site-packages/py4j/java_gateway.py", line 1217, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/home/igor/miniconda3/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

Exception in thread read_grpc_client_inputs:
Traceback (most recent call last):
  File "/home/igor/miniconda3/lib/python3.11/threading.py", line 1038, in _bootstrap_inner
Exception in thread read_grpc_client_inputs:
Traceback (most recent call last):
  File "/home/igor/miniconda3/lib/python3.11/threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "/home/igor/miniconda3/lib/python3.11/threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "/home/igor/miniconda3/lib/python3.11/site-packages/apache_beam/runners/worker/data_plane.py", line 669, in <lambda>
    target=lambda: self._read_inputs(elements_iterator),
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/igor/miniconda3/lib/python3.11/site-packages/apache_beam/runners/worker/data_plane.py", line 652, in _read_inputs
    for elements in elements_iterator:
  File "/home/igor/miniconda3/lib/python3.11/site-packages/grpc/_channel.py", line 542, in __next__
    return self.

5> +I[2866735578586270930]
3> +I[-5977226134882793068]
9> +I[-214587660841360983]
4> +I[-5544873727434937957]
10> +I[-8752139827143782728]
2> +I[-4329604452233575483]
1> +I[-6628258432766251696]
8> +I[7561179697052145462]
6> +I[3272481767206754291]
7> +I[-4193229494577812445]


In [1]:
import pandas as pd

In [3]:
df1_db = pd.read_csv("DBLP.csv", sep=",", encoding="utf-8", keep_default_na=False)
df2_sc = pd.read_csv("Scholar.csv", sep=",", encoding="utf-8", keep_default_na=False)

In [4]:
df1_db["source"] = "dblp"
df1_db

,id,title,authors,venue,year,source
0,conf/vldb/RusinkiewiczKTWM95,Towards a Cooperative Transaction Model - The ...,"M Rusinkiewicz, W Klas, T Tesch, J Wδsch, P Muth",VLDB,1995,dblp
1,journals/sigmod/EisenbergM02,SQL/XML is Making Good Progress,"A Eisenberg, J Melton",SIGMOD Record,2002,dblp
2,conf/vldb/AmmannJR95,Using Formal Methods to Reason about Semantics...,"P Ammann, S Jajodia, I Ray",VLDB,1995,dblp
3,journals/sigmod/Liu02,Editor's Notes,L Liu,SIGMOD Record,2002,dblp
4,journals/sigmod/Hammer02,Report on the ACM Fourth International Worksho...,N/A,N/A,2002,dblp
...,...,...,...,...,...,...
2611,conf/vldb/ShuklaDNR96,Storage Estimation for Multidimensional Aggreg...,"A Shukla, P Deshpande, J Naughton, K Ramasamy",VLDB,1996,dblp
2612,journals/sigmod/Aberer03,Call for Book Reviews,N/A,N/A,2003,dblp
2613,conf/vldb/RamakrishnanR96,Modeling Design Versions,"R Ramakrishnan, D Ram",VLDB,1996,dblp
2614,conf/vldb/ShaferAM96,SPRINT: A Scalable Parallel Classifier for Dat...,"J Shafer, R Agrawal, M Mehta",VLDB,1996,dblp


In [5]:
df2_sc["source"] = "scholar"
df2_sc

,id,title,authors,venue,year,source
0,aKcZKwvwbQwJ,11578 Sorrento Valley Road,QD Inc,"San Diego,",,scholar
1,ixKfiTHoaDoJ,Initiation of crazes in polystyrene,"AS Argon, JG Hannoosh","Phil. Mag,",,scholar
2,3BxllB4wwcIJ,Immunogold labelling is a quantitative method ...,"GH Hansen, LL Wetterberg, H SjÃ¶strÃ¶m, O NorÃ©n","The Histochemical Journal,",1992,scholar
3,d2WWxwKMex4J,The Burden of Infectious Disease Among Inmates...,"TM Hammett, P Harmon, W Rhodes",see,,scholar
4,cZCX-AQpjccJ,The Role of Faculty Advising in Science and En...,JR Cogdell,"NEW DIRECTIONS FOR TEACHING AND LEARNING,",1995,scholar
...,...,...,...,...,...,...
64258,TX4SGKJQUiEJ,Theophylline toxicity.,DS Cooling,"Journal of Emergency Medicine,",1993,scholar
64259,url:http://portal.acm.org/citation.cfm%3Fid%3D...,Source ACM Transactions on Database Systems (T...,RW Taylor,"ACM Transactions on Database Systems (TODS),",1983,scholar
64260,7Hi3KKsxKNwJ,Range Nesting: A Fast Method to Evaluate Quant...,"M Jarke, J Koch",,,scholar
64261,url:http://portal.acm.org/citation.cfm%3Fid%3D...,Source ACM Transactions on Database Systems (T...,W Kim,"ACM Transactions on Database Systems (TODS),",1993,scholar


In [6]:
df3 = pd.concat([df1_db, df2_sc], axis=0)
df3

,id,title,authors,venue,year,source
0,conf/vldb/RusinkiewiczKTWM95,Towards a Cooperative Transaction Model - The ...,"M Rusinkiewicz, W Klas, T Tesch, J Wδsch, P Muth",VLDB,1995,dblp
1,journals/sigmod/EisenbergM02,SQL/XML is Making Good Progress,"A Eisenberg, J Melton",SIGMOD Record,2002,dblp
2,conf/vldb/AmmannJR95,Using Formal Methods to Reason about Semantics...,"P Ammann, S Jajodia, I Ray",VLDB,1995,dblp
3,journals/sigmod/Liu02,Editor's Notes,L Liu,SIGMOD Record,2002,dblp
4,journals/sigmod/Hammer02,Report on the ACM Fourth International Worksho...,N/A,N/A,2002,dblp
...,...,...,...,...,...,...
64258,TX4SGKJQUiEJ,Theophylline toxicity.,DS Cooling,"Journal of Emergency Medicine,",1993,scholar
64259,url:http://portal.acm.org/citation.cfm%3Fid%3D...,Source ACM Transactions on Database Systems (T...,RW Taylor,"ACM Transactions on Database Systems (TODS),",1983,scholar
64260,7Hi3KKsxKNwJ,Range Nesting: A Fast Method to Evaluate Quant...,"M Jarke, J Koch",,,scholar
64261,url:http://portal.acm.org/citation.cfm%3Fid%3D...,Source ACM Transactions on Database Systems (T...,W Kim,"ACM Transactions on Database Systems (TODS),",1993,scholar


In [7]:
df3 = df3.astype(str)

In [8]:
df3.to_csv("df__full_merged.csv",index=False)

In [ ]:
import pandas as pd
df_xml = pd.read_xml("/home/igor/Downloads/dblp.xml")
df_xml


: 

In [1]:
import pandas as pd

In [3]:
df_txt = pd.read_table("/home/igor/Downloads/ncvoter_Statewide.txt",encoding='unicode_escape')

: 

In [ ]:
df_txt